In [1]:
class MyClass
  def my_method(my_arg)
    my_arg * 2
  end
end

obj = MyClass.new
obj.my_method(3)

6

In [2]:
obj.send(:my_method, 3)

6

为什么要用`send`方法，而不用原先的点标识符呢？这个是因为在`send`方法里，你想调用的方法名编程了参数，这样就可以在代码运行的最后一刻决定调用哪个方法。这个技巧叫做**动态转发**。

In [4]:
require 'pry'

false

In [10]:
class Pry
def rf(options = {})
  defaults = {}
  attributes = [:input, :output, :commands, :print, :quiet, :execution_handler, :hooks, :custom_completions, :prompt, :memory_size, :extra_sticky_locals]
  
  attributes.each do |attribute|
    defaults[attribute] = Pry.send attribute
  end
  
  defaults.merge!(options).each do |key, value|
    send("#{key}=", value) if respond_to?("#{key}=")
  end
  true
end
end

:refresh

这段代码用`send`方法把每个属性的默认值放入一张哈希表中，然后把这张哈希表和传入参数的`options`合并。最后使用`send`方法调用每个属性的写方法（如`memery_size=`)。`Kernel#respond_to?`方法检测注入`Pry#memory_size=`这样的方法是否存在，如果存在则返回`true`。这样，如果参数`options`哈希表中设置了当前属性中不存在的属性，这些属性就会被忽略。

你可以用`Module#define_method()`方法随时定义一个方法，只需要提供给一个方法名和充当方法主体的块：

In [26]:
class MyClass
  define_method :my_method do |my_arg|
    my_arg * 3
  end
end

obj = MyClass.new
obj.my_method(2)

6

`defind_method`方法在`MyClass`内部执行，因此`my_method`定义为`MyClass`的实例方法。这种在运行时定义方法的技术称为动态方法

In [27]:
class Computer
  def initialize(computer_id, data_source)
    @id = computer_id
    @data_source = data_source
  end
  
  def mouse
    component :mouse
  end
  
  def cpu
    component :cpu
  end
  
  def keyboard
    component :keyboard
  end
  
  def component(name)
    info = @data_source.send "get_#{name}_info", @id
    price = @data_source.send "get_#{name}_price", @id
    result = "#{name.capitalize}: #{info} ($#{price})"
    return "* #{result}" if price >= 100
    result
  end
end

:component

In [29]:
my_computer = Computer.new(42, DS.new)
my_computer.cpu

NameError: uninitialized constant DS

In [31]:
class Computer
  def initialize(computer_id, data_source)
    @id = computer_id
    @data_source = data_source
  end
  
  def self.define_component(name)
    define_method(name) do
      info = @data_source.send "get_#{name}_info", @id
      price = @data_source.send "get_#{name}_price", @id
      result = "#{name.capitalize}: #{info} ($@{price})"
      return "* #{result}" if price >= 100
      result
    end
  end
  
  define_component :mouse
  define_component :cpu
  define_component :keyboard
end

:keyboard